In [9]:
%%writefile CombiningTree.go
//incorrect

package main

import (
	"container/list"
	"math"
	"time"
    "sync"
)

type CStatus int

const (
	IDLE CStatus = iota
	FIRST
	SECOND
	RESULT
	ROOT
)

type Node struct {
	parent      *Node
	cStatus     CStatus
	locked      bool
	result      int
	firstValue  int
	secondValue int
	release     chan bool
}

func newNode(myParent Node) Node {
	n := Node{parent: &myParent}
	n.cStatus = IDLE
	n.locked = false
	n.result = 0
	n.firstValue = 0
	n.secondValue = 0
	n.release = make(chan bool)
	return n
}

func precombine(node Node) bool {
	if node.locked {
		<-node.release
	}
	switch node.cStatus {
	case IDLE:
		node.cStatus = FIRST
		return true
	case FIRST:
		node.locked = true
		node.cStatus = SECOND
		return false
	case ROOT:
		return false
	default:
		println("unexpected Node state in precombine")
		return false // error
	}
}

func combine(combined int, node Node) int {
	if node.locked {
		<-node.release
	}
	node.locked = true
	node.firstValue = combined
	switch node.cStatus {
	case FIRST:
		return node.firstValue
	case SECOND:
		return node.firstValue + node.secondValue
	default:
		println("unexpected Node state in combine")
		return -1 // error

	}
}

func op(combined int, node Node) int {
	switch node.cStatus {
	case ROOT:
		prior := node.result
		node.result += combined
		return prior
	case SECOND:
		node.secondValue = combined
		node.locked = false
		node.release <- true // wake up waiting threads
		if node.cStatus != RESULT {
			<-node.release
		}
		node.locked = false
		node.release <- true
		node.cStatus = IDLE
		return node.result
	default:
		println("unexpected Node state in op")
		return -1 // error

	}
}

func distribute(prior int, node Node) {
	switch node.cStatus {
	case FIRST:
		node.cStatus = IDLE
		node.locked = false
		break
	case SECOND:
		node.result = prior + node.firstValue
		node.cStatus = RESULT
		break
	default:
		println("unexpected Node state in distribute")
	}
	node.release <- true
}

type CombinningTree struct {
	nodes []Node
	leaf  []Node
}

func newTree(width int) CombinningTree {

	tree := CombinningTree{}
	tree.nodes = make([]Node, 2*width-1)
	tree.leaf = make([]Node, width)

	length := 2*width - 1
	tree.nodes[0] = newNode(Node{parent: nil, cStatus: ROOT})
	for i := 1; i < length; i++ {
		tree.nodes[i] = newNode(tree.nodes[(i-1)/2])
	}
	for i := 1; i < width; i++ {
		tree.leaf[i] = tree.nodes[length-i-1]
	}
	return tree
}

func getAndIncrement(tree CombinningTree, id int) int {
	stack := list.New()
	myLeaf := tree.leaf[id%len(tree.leaf)]
	println(id, len(tree.leaf))
	node := myLeaf
	for node.parent != nil && precombine(node) {
		node = *node.parent
	}
	stop := node
	combined := 1
	for node = myLeaf; node != stop; node = *node.parent {
		combined = combine(combined, node)
		stack.PushBack(node)
	}

	prior := op(combined, stop)
	for stack.Len() > 0 {

		node = stack.Remove(stack.Back()).(Node)
		distribute(prior, node)
	}
	return prior

}

var TH int
var NUM int

func main() {
	TH = 12
	NUM = 100
	width := int(math.Ceil(float64(TH / 2)))
	tree := newTree(width)
	print(width)
	print("-leaves Combining tree.\n")
	print("Starting ")
	print(TH)
    print(" threads doing increments ...\n")
    var wg sync.WaitGroup
    
	start := time.Now()
	for index := 0; index < TH; index++ {
        wg.Add(1)
		go func(id int) {
			for i := 0; i < NUM; i++ {
				getAndIncrement(tree, id)
			}
		}(index)
	}
    
   wg.Wait()
	stop := time.Now()
	diff := stop.Sub(start)
	print("Total: ")
	print(tree.nodes[0].result)
	print("\nTotal time: ")
	print(diff)
	print(" s")
	
}


Overwriting CombiningTree.go


In [11]:
!go run CombiningTree.go

'go' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
%%writefile CounterNetwork.go

package main

import "math/rand"
import "time"
import "fmt"
import "sync"

type Balancer struct {
	toggle bool
	lock   sync.Mutex
}

func balancerTraverse(b *Balancer) int {
	b.lock.Lock()
	fmt.Println(b.toggle)
	defer func() { b.toggle = !b.toggle }()
	defer b.lock.Unlock()
	if b.toggle {
		return 0
	} else {
		return 1
	}
}

type Merger struct {
	width int
	layer []Balancer
	half  []Merger
}

func newMerger(myWidth int) *Merger {
	m := Merger{}
	m.width = myWidth
	m.layer = make([]Balancer, myWidth/2)
	for i := 0; i < m.width/2; i++ {
		m.layer[i] = Balancer{toggle: true}
	}
	if m.width > 2 {
		m.half = []Merger{*newMerger(myWidth / 2), *newMerger(myWidth / 2)}
	}
	return &m
}

func mergerTraverse(input int, m *Merger) int {
	output := 0
	if m.width <= 2 {
		return balancerTraverse(&m.layer[0])
	}
	if input < m.width/2 {
		output = mergerTraverse(input/2, &m.half[input%2])
	} else {
		output = mergerTraverse(input/2, &m.half[1-(input%2)])
	}
	return (2 * output) + balancerTraverse(&m.layer[output])

}

type Bitonic struct {
	half   []Bitonic
	merger Merger
	width  int
}

func newBitonic(myWidth int) *Bitonic {
	bit := Bitonic{}
	bit.width = myWidth
	bit.merger = *newMerger(myWidth)
	if myWidth > 2 {
		bit.half = []Bitonic{*newBitonic(myWidth / 2), *newBitonic(myWidth / 2)}
	}
	return &bit
}

func bitonicTraverse(input int, bit *Bitonic) int {
	output := 0
	subnet := input / (bit.width / 2)
	if bit.width > 2 {
		output = bitonicTraverse(input/2, &bit.half[subnet])
	}
	if input >= bit.width/2 {
		return mergerTraverse(bit.width/2+output, &bit.merger)
	} else {
		return mergerTraverse(output, &bit.merger)
	}
}

func main() {
	width := 4
	counters := make([]int, width)
	bitonic := newBitonic(width)
	tokenCount := 10
	tokens := make([]int, tokenCount)

	for i := 0; i < tokenCount; i++ {
		tokens[i] = rand.Intn(width)
	}
	var wg sync.WaitGroup

	start := time.Now()
	for idx := 0; idx < tokenCount; idx++ {
		wg.Add(1)
		go func(i int) {
			counters[bitonicTraverse(tokens[i], bitonic)] += 1
		}(idx)
	}

	time.Sleep(10000)
	// wg.Wait()
	stop := time.Now()
	diff := stop.Sub(start)

	for i := 0; i < width; i++ {
		fmt.Println("output: ", i, ", Count: ", counters[i])
	}
	fmt.Println("Total time to traverse the network: ", diff)
}


Writing CounterNetwork.go
